# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import json
import os
os.chdir('../config')
from config import gp
os.chdir('../WeatherPy') 
# Import API key
gmaps.configure(api_key=gp)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
load_csv = "../WeatherPy/output/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Current Temp,Wind Speed
0,isangel,40,VU,1579589751,65,-19.55,169.27,82.40,82.40,6.93
1,zharkent,57,KZ,1579589751,85,44.16,80.00,21.97,21.97,2.48
2,tuy hoa,11,VN,1579589690,78,13.08,109.30,75.74,75.74,15.61
3,hermanus,22,ZA,1579589751,93,-34.42,19.23,73.00,68.59,10.00
4,sept-iles,5,CA,1579589752,69,50.20,-66.38,-7.60,-7.60,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
locations = city_df[["Latitude", "Longitude"]]
humidity = city_df['Humidity'].astype(int)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
# max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Current Temp,Wind Speed
0,41,cape town,0,ZA,1579589757,60,-33.93,18.42,73.40,72.77,6.93
1,261,sur,0,OM,1579589788,47,22.57,59.53,72.23,72.23,1.50
2,474,port lincoln,0,AU,1579589818,50,-34.73,135.87,72.61,72.61,9.15
3,485,san patricio,0,MX,1579589819,72,19.22,-104.70,79.11,79.11,5.41
4,535,kantabanji,0,IN,1579589827,45,20.48,82.92,78.28,78.28,4.23
5,557,dabhol,0,IN,1579589830,65,17.60,73.17,77.41,77.41,9.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# create hotel_df with hotel name column
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df=hotel_df.rename(columns={"Latitude":"Lat","Longitude":"Lng"})
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Current Temp,Wind Speed,Hotel Name
0,41,cape town,0,ZA,1579589757,60,-33.93,18.42,73.40,72.77,6.93,NaN
1,261,sur,0,OM,1579589788,47,22.57,59.53,72.23,72.23,1.50,NaN
2,474,port lincoln,0,AU,1579589818,50,-34.73,135.87,72.61,72.61,9.15,NaN
3,485,san patricio,0,MX,1579589819,72,19.22,-104.70,79.11,79.11,5.41,NaN
4,535,kantabanji,0,IN,1579589827,45,20.48,82.92,78.28,78.28,4.23,NaN
5,557,dabhol,0,IN,1579589830,65,17.60,73.17,77.41,77.41,9.44,NaN


In [37]:
#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
#checker variable
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gp
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Current Temp,Wind Speed,Hotel Name
0,41,cape town,0,ZA,1579589757,60,-33.93,18.42,73.40,72.77,6.93,Southern Sun Waterfront Cape Town
1,261,sur,0,OM,1579589788,47,22.57,59.53,72.23,72.23,1.50,Sur Plaza Hotel
2,474,port lincoln,0,AU,1579589818,50,-34.73,135.87,72.61,72.61,9.15,Port Lincoln Hotel
3,485,san patricio,0,MX,1579589819,72,19.22,-104.70,79.11,79.11,5.41,Hotel Cabo Blanco
4,535,kantabanji,0,IN,1579589827,45,20.48,82.92,78.28,78.28,4.23,Kamdhenu Borewell And Trading
5,557,dabhol,0,IN,1579589830,65,17.60,73.17,77.41,77.41,9.44,Ashwamedh Resort


In [38]:
#clean up DataSet by remooving NAN
hotel_df_clean = hotel_df.dropna()
hotel_df_clean
hotel_df=hotel_df_clean
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Current Temp,Wind Speed,Hotel Name
0,41,cape town,0,ZA,1579589757,60,-33.93,18.42,73.40,72.77,6.93,Southern Sun Waterfront Cape Town
1,261,sur,0,OM,1579589788,47,22.57,59.53,72.23,72.23,1.50,Sur Plaza Hotel
2,474,port lincoln,0,AU,1579589818,50,-34.73,135.87,72.61,72.61,9.15,Port Lincoln Hotel
3,485,san patricio,0,MX,1579589819,72,19.22,-104.70,79.11,79.11,5.41,Hotel Cabo Blanco
4,535,kantabanji,0,IN,1579589827,45,20.48,82.92,78.28,78.28,4.23,Kamdhenu Borewell And Trading
5,557,dabhol,0,IN,1579589830,65,17.60,73.17,77.41,77.41,9.44,Ashwamedh Resort


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [54]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map and display
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info )

# Add layer
fig.add_layer(markers)

# Display figure
fig
# markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(markers)
# fig

# Display Map

Figure(layout=FigureLayout(height='420px'))